In [3]:
from graphdatascience import GraphDataScience
from neo4j import GraphDatabase
from yfiles_jupyter_graphs import GraphWidget

In [4]:
uri = "bolt://localhost:7687" 
user = "neo4j"  
password = "12345678"
driver = GraphDatabase.driver(uri=uri,auth=(user,password),database='neo4j')
session = driver.session(database='neo4j')

### Visualize the graph

In [5]:

result = session.run("CALL db.schema.visualization()")
w = GraphWidget(graph = result.graph())
w.show()

GraphWidget(layout=Layout(height='500px', width='100%'))

In [80]:
session.run('''
CALL gds.graph.project(
  'ratings',
  ['User','Movie'],
  {
    RATED: {
      orientation: 'UNDIRECTED',
      properties: 'rating'
    }
  }
)
''')

In [81]:
session.run('''
CALL gds.fastRP.mutate('ratings',
  {
    embeddingDimension: 256,
    randomSeed: 7474,
    mutateProperty: 'embedding',
    relationshipWeightProperty: 'rating',
    iterationWeights: [0.8, 1, 1, 1]
  }
)
YIELD nodePropertiesWritten
''')

In [82]:
result = session.run('''
    CALL gds.knn.write('ratings', {
    topK: 5,
    nodeProperties: ['embedding'],
    randomSeed: 42,
    concurrency: 1,
    sampleRate: 1.0,
    deltaThreshold: 0.0,
    writeRelationshipType: "SIMILAR",
    writeProperty: "score"
})
YIELD nodesCompared, relationshipsWritten, similarityDistribution
RETURN nodesCompared, relationshipsWritten, similarityDistribution.mean as meanSimilarity
''').to_df()
print(result)

   nodesCompared  relationshipsWritten  meanSimilarity
0          10440                 52200        0.950769


In [83]:
similar_user = session.run('''
    MATCH (n:User)-[r:SIMILAR]->(m:User)
    RETURN n.id as person1, m.id as person2, r.score as similarity
    ORDER BY similarity DESCENDING, person1, person2 limit 100
''').to_df()
print(similar_user)

    person1  person2  similarity
0   user126  user379    0.989860
1   user379  user126    0.989860
2   user242  user468    0.987524
3   user468  user242    0.987524
4   user130  user574    0.985089
..      ...      ...         ...
95  user235  user229    0.978445
96  user404  user411    0.978418
97  user411  user404    0.978418
98  user130  user242    0.978277
99  user394  user379    0.978269

[100 rows x 3 columns]


### User-based Recommendation

In [13]:
query = '''
MATCH (u:User {id: 'user470'})
MATCH (u)-[r1:SIMILAR]->(similarUser)-[r2:RATED]->(m:Movie)
WHERE NOT EXISTS((u)-[:RATED]->(m))
RETURN u.id AS userId, m.title AS movieTitle, m.id AS movieId, COLLECT(similarUser.id) AS similarUserIds
LIMIT 10
'''
result = session.run(query)
print(result.to_df())
# widget = GraphWidget(graph = result.graph(), overview_enabled=True)
# widget.show()

    userId                     movieTitle movieId  \
0  user470  Independence Day (a.k.a. ID4)     780   
1  user470        American President, The      11   
2  user470                          Congo     160   
3  user470              Santa Clause, The     317   
4  user470                      Coneheads     435   
5  user470      Silence of the Lambs, The     593   
6  user470                     Home Alone     586   
7  user470                     Piano, The     509   
8  user470                   Philadelphia     508   
9  user470                Made in America     489   

                       similarUserIds  
0                  [user347, user179]  
1  [user347, user347, user94, user94]  
2                   [user347, user94]  
3          [user347, user94, user179]  
4          [user347, user229, user94]  
5          [user229, user94, user179]  
6                   [user229, user94]  
7                           [user229]  
8                           [user229]  
9               

### Item-based Recommendation

In [16]:
query = '''
MATCH (targetMovie:Movie {id: '586'})
MATCH (targetMovie)-[similarity:SIMILAR]-(similarMovie:Movie)<-[:IS_GENRE_OF]-(genre:Genre)
WHERE similarMovie.id <> targetMovie.id
RETURN DISTINCT similarMovie.id AS MovieId, COLLECT(DISTINCT genre.name) AS genres, similarMovie.title AS MovieName , similarity.score AS score
ORDER BY similarity.score DESC
LIMIT 10
'''
result = session.run(query)
print(result.to_df())
# widget = GraphWidget(graph = result.graph())
# widget.show()

  MovieId                                  genres            MovieName  \
0     500                         [Comedy, Drama]       Mrs. Doubtfire   
1     597                       [Comedy, Romance]         Pretty Woman   
2     367        [Action, Comedy, Crime, Fantasy]            Mask, The   
3     364  [Adventure, Animation, Drama, Musical]       Lion King, The   
4     480   [Action, Adventure, Sci-Fi, Thriller]        Jurassic Park   
5   52715                                [Comedy]  Kickin It Old Skool   

      score  
0  0.977436  
1  0.974219  
2  0.973562  
3  0.973017  
4  0.972254  
5  0.881000  
